In [318]:
# Creating a new ABM using a network structure

In [319]:
# loading packages
import random
import numpy as np
import networkx as nx
import pandas as pd
from collections import Counter
import itertools

import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import StagedActivation, BaseScheduler
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.datacollection import DataCollector

# CPS workflow

## World 


## Agents
Universal parameters:
problem_solving
communication

role_correct
role_distribution
role_negotiation lvl3
role_monitoring


Level 1

Level 2

Level 3
role_distributed=1

Level 4
Role_effect=1





## Teams are formed

## Problems
All team is posed with a problem with Task_complexity (1:3) and Size (1000)


Then we distribute role: 1 to len(team) (Do you fulfill your role or not?)
- each is assigned a "correct" role
- Then roles are more probable to become correct based on lvl2-4

- If Lvl 3/4 then roles are distributed correctly/evenly at first
- Self.Role=X

First we try to generate common-ground:
- Only lvl 2-4
- Based on agents Communication parameter
- Common_ground = 0 or 1

Then we simulate role_alignment:
- There some 0.8 prob that role stays correct
- Theres some 0.2 prob that role might shift to incorrect

Then Solutions are generated:
- generate some 50 to 150 solution (based on problem_solving)
- If lvl doesnt match task lvl then solution cannot be given

If some role > 1:
- Then conflict appears

If common_ground=0
- Lowest solution is chosen

If all roles are evenly distributed and common ground = 1:
    Then choose the highest solution

Adjustement of error/Meta negotiations/Self and team monitoring:
- Resolve_conflict
    - 50% prob of conflict resolvement
    - 90% if lvl 3 is present
- Reach_common_ground
    - 


In [320]:
# Simulating agents parameters

# generating pisa mean scores
def generate_pisa_score():
    score = np.random.normal(500, 95)
    return score

# generating cps levels
def generate_cps_level(x):
   # if x is below 340 then cps level is 0
   # if x is between 340 and 440 then cps level is 1
    # if x is between 440 and 540 then cps level is 2
    # if x is between 540 and 640 then cps level is 3
    # if x is above 640 then cps level is 4
    x = np.random.normal(500, 95)
    if x < 340:
        cps_level = 0
    elif x >= 340 and x < 440:
        cps_level = 1
    elif x >= 440 and x < 540:
        cps_level = 2
    elif x >= 540 and x < 640:
        cps_level = 3
    elif x >= 640:
        cps_level = 4
    return cps_level

# generating cps levels
def generate_cps_level_discrete():
   # generate a 0 with a 6% probability 
   # a 1 with a 22% probability 
   # and a 2 with a 36% probability
   # and a 3 with a 28% probability
   # and a 4 with a 8% probability
    return np.random.choice([0,1,2,3,4], p=[0.06, 0.22, 0.36, 0.28, 0.08])

def cps_weight(cps_level):
    if cps_level == 0:
        weight = random.uniform(0.5, 1)
    elif cps_level == 1:
        weight = random.uniform(1, 1.25)
    elif cps_level == 2:
        weight = random.uniform(1.25, 1.5)
    elif cps_level == 3:
        weight = random.uniform(1.75, 2)
    elif cps_level == 4:
        weight = random.uniform(2, 2.25)
    
    return weight

# Generating a function which creates a task work level which is a random number between 0 and 1 and then multiplied by x and add uncertainty around this estimate of 0.77

# finding agents with the same role
def get_agents_with_same_role(team_current_roles, team_id, agent_id):
    role = team_current_roles[team_id][agent_id]
    agents_with_same_role = [agent for agent, agent_role in team_current_roles[team_id].items() if agent_role == role]
    return agents_with_same_role

# unnest list in list function
def unnest_list(nested_list):
    result = []
    for item in nested_list:
        if isinstance(item, list):
            result.extend(unnest_list(item))
        else:
            result.append(item)
    return result

# remove duplicates
def remove_duplicates(original_list):
    result_list = []
    for item in original_list:
        if item not in result_list:
            result_list.append(item)
    return result_list

# Custom Staged activation

In [321]:
from mesa.time import StagedActivation

class CustomStagedActivation(StagedActivation):
    def __init__(self, model, stage_list, num_rounds, shuffle=False, shuffle_between_stages=False):
        super().__init__(model, stage_list, shuffle, shuffle_between_stages)
        self.num_rounds = num_rounds
        self.p_round = 0

    def step(self):
        """Executes all the stages for all agents, including a loop alternating between stages two and three."""
        # To be able to remove and/or add agents during stepping
        # it's necessary to cast the keys view to a list.
        agent_keys = list(self._agents.keys())
        if self.shuffle:
            self.model.random.shuffle(agent_keys)
        for stage in self.stage_list:
            self.p_round = 0
            if stage == "stage_one":
                for agent_key in agent_keys:
                    if agent_key in self._agents:
                        getattr(self._agents[agent_key], 'stage_one')()
            elif stage == "stage_two":
                for _ in range(self.num_rounds):  # Loop for stage two and three (num_rounds times)
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], 'stage_two')()  # Run stage
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], "stage_three")()  # Run stage
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], "stage_four")()
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], "stage_five")()
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], "stage_six")()
                    
                    self.p_round += 1
            elif stage == "stage_seven":
                for agent_key in agent_keys:
                    if agent_key in self._agents:
                        getattr(self._agents[agent_key], "stage_seven")()  # Run stage
            # We recompute the keys because some agents might have been removed
            # in the previous loop.
            agent_keys = list(self._agents.keys())
            if self.shuffle_between_stages:
                self.model.random.shuffle(agent_keys)
            self.time += self.stage_time

        self.steps += 1


# TeamAgent

In [381]:
class TeamAgent(Agent):
    """ An agent with X."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.pisa_score = generate_pisa_score()
        self.cps_level = generate_cps_level(self.pisa_score)
        self.cps_weight = cps_weight(self.cps_level)
        
        self.team_members = []
        self.team_id = None
        self.team_cps = []
        
        self.current_role = None
        self.correct_role = None
        self.role_disagreement = 0  # can be 0 or 1
        self.is_role_correct = 0
        self.agents_with_same_role = []

        self.solution = 0

        self.common_ground = 0

        

    # Preparing agents into teams
    def reset_team(self):
        self.team_members = []
        self.team_id = None


    ## Joining teams
    def join_teams3(self):
        self.team_id = self.model.agent_team_dict.get(self.unique_id)

        #other_agent_ids = [agent_id for agent_id, t_id in self.model.agent_team_dict.items() if t_id == self.team_id and agent_id != self.unique_id]
        #self.team_members = other_agent_ids + [self.unique_id]
        members = [agent_id for agent_id, t_id in self.model.agent_team_dict.items() if t_id == self.team_id]
        self.team_members = members

        print("agent_id:", self.unique_id, "Team_id:", self.team_id, "Team_members:", self.team_members)

    ## Creating a correct role
    def create_roles(self):

        x = list( range(1, len(self.model.teams[self.team_id])) )
        self.model.teams[self.team_id].append()
    

        self.model.teams[self.team_id]

    def get_correct_role(self):
        
        for i in self.team_members:
            self.correct_role = self.team_members.index(self.unique_id)

        print("Your correct role is:", self.correct_role)
    

    def grab_team_cps(self):
        self.team_cps = []

        for i in self.team_members:
            self.team_cps.append( self.model.schedule.agents[i].cps_level )

        print("This is your own cps:", self.cps_level)
        print("This is your weight", self.cps_weight)
        print("This is the cps of whole team:", self.team_cps)
        
    # Defining the problem loop where evaluation takes place

    def get_start_role(self):
        alist = list(range(len(self.team_members)))
        alist.remove(self.correct_role)
        other_roles = alist


        if 3 in self.team_cps or 4 in self.team_cps:
            self.current_role = self.correct_role # should change this to just be the prob of choice
            print("First round/correct_role_assigned.", "agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
        else:
            self.current_role = np.random.choice(other_roles)
            print("First round/random_role_assigned.", "agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
        
        print("Your starting role is:", self.current_role)
        print("Your correct role is:", self.correct_role)
        

    def get_current_role(self):
        # Simulate a current_role for each agent from 0 to length of team_members the curren_role should have a 80% probability of being correct_role and a 20% probability of being a random role
        alist = list(range(len(self.team_members))).copy()
        alist.remove(self.correct_role)
        other_roles = alist

        alist2 = list(range(len(self.team_members))).copy()
        print("length of self team mebers", len(self.team_members) )
        print("alist2:", alist2, "current_role:", self.current_role, "Agent:", self.unique_id, "team:", self.team_id, "alist:", alist)
        alist2.remove(self.current_role)
        other_roles2 = alist2

        # propability list where the correct role has a 80% probability and the other roles have a 20% probability
        ## if all cps are below 3 then the probability of the role which was taken in the previous round is 50% and the probability of all other roles is 50% / number of other roles
        prob_role_disagreement_true = [0.5] + [0.5 / len(other_roles2)] * len(other_roles2)
        # if all 
        prob_role_disagreement_false = [0.75] + [0.25 / len(other_roles2)] * len(other_roles2)
        p_role_high = [0.95] + [0.05 / len(other_roles)] * len(other_roles)
        
        

        if all(self.team_cps) < 3:
            if self.role_disagreement == 1:
                self.current_role = np.random.choice([self.current_role] + other_roles2, p=prob_role_disagreement_true)
                print("Role Disagreement." ,"agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
            else:
                self.current_role = np.random.choice([self.current_role] + other_roles2, p=prob_role_disagreement_false)
                print("No Role Disagreement." ,"agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
        elif 3 in self.team_cps and all(self.team_cps) < 4 and self.cps_level == 3:
            self.current_role = np.random.choice([self.correct_role] + other_roles, p=p_role_high)
            print("Level 3 and no 4s in team.", "agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
        elif 3 in self.team_cps and all(self.team_cps) < 4 and self.cps_level < 3:
            self.current_role = np.random.choice([self.correct_role] + other_roles, p=prob_role_disagreement_false)
            print("Level 3 and no 4s in team.", "agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
        elif 4 in self.team_cps:
            self.current_role = np.random.choice([self.correct_role] + other_roles, p=p_role_high)
            print("Level 4 in team.", "agent", self.unique_id, "team", self.team_id, "This is my role:", self.current_role)
    
        
        print("Your current role is:", self.current_role)
        print("Your correct role is:", self.correct_role)

    def getting_roles(self):
            # creating the roles
        if self.model.schedule.p_round == 0:
            self.get_start_role()
        else:
            self.get_current_role()
        
        if self.team_id in self.model.team_current_roles:
            if self.unique_id in self.model.team_current_roles[self.team_id]:
                self.model.team_current_roles[self.team_id][self.unique_id].append(self.current_role)
            else:
                self.model.team_current_roles[self.team_id][self.unique_id] = [self.current_role]
        else:
            self.model.team_current_roles[self.team_id] = {self.unique_id: [self.current_role]}
            
        print("This is the current roles of my team:", self.model.team_current_roles[self.team_id])

    # finding out if theres been 2 people in the same role
    def find_role_disagreement(self):

        for i in self.model.team_current_roles[self.team_id]:
            if self.model.team_current_roles[self.team_id].count(i) > 1:
                self.role_disagreement = 1
            else:
                self.role_disagreement = 0
        
        
        # find who is disagreeing
    def find_agents_with_same_role(self):
        
        agents_with_same_role_list = [agent for agent, agent_role in self.model.team_current_roles[self.team_id].items() if agent_role == self.current_role]
        
        self.agents_with_same_role = agents_with_same_role_list

        print("this is the current roles of my team:", self.model.team_current_roles[self.team_id])
        print("this is the agents with same role", self.agents_with_same_role)

    
    
       
    def establish_common_ground(self):
        # common ground might be = 1 with a default prob of 50% however if there is a cps_lvl 2 in the team_members then add the weighted probability of cps_lvl

        prob = 0.5
        for i in self.team_cps:
            if i > 1:
                prob *= (1.04 ** i) # The combination of "to the power of" and 1.05 creates a good balanced probability of synergy based on the incremental charecteristics
        
        # generate a common ground 0 or 1 for each agent
        common_ground = np.random.binomial(1, prob)

        # append this to a dictionary with team as key and common_ground as value if team is not in the dictionary already
    
        if self.team_id not in list( self.model.team_common_ground.keys() ):
            self.model.team_common_ground[self.team_id] = [common_ground]

        self.common_ground = common_ground
        
        print("This is the common ground for", self.team_id, " :", self.model.team_common_ground[self.team_id])
        

    # this loop creates the problem, the roles, common ground etc.
    def problem_loop(self):

        # create a problem_solving round counter
        

        # If the task fits level
        if self.cps_level >= self.model.problem_complexity:
            self.solution = random.uniform(0.9, 1.1) * self.pisa_score # 10% uncertainty around the solution because of the gap in cps level point range
        # If the task is too complex
        elif self.cps_level < self.model.problem_complexity:
            self.solution = 0 # FOR NOW! this will remain 0 I might change this to a prob 
        
        # add the self solution to a model class variable which is indexed by the team_id and agent_id
         
        if self.team_id in self.model.team_solutions:
            if self.unique_id in self.model.team_solutions[self.team_id]:
                self.model.team_solutions[self.team_id][self.unique_id].append(self.solution)
            else:
                self.model.team_solutions[self.team_id][self.unique_id] = [self.solution]
        else:
            self.model.team_solutions[self.team_id] = {self.unique_id: [self.solution]}

        print("agent", self.unique_id, "team", self.team_id, "This is my solution:", self.solution)
        print("This is the team solutions:", self.model.team_solutions)

    # conflict might arise  
    def conflict(self):
        
        if self.team_id not in self.model.team_conflict:
            
            team_current_roles = list( self.model.team_current_roles[self.team_id] )
            
            all_agents_in_disagreement = 0 # creates a count of how many agent who share a role
            for i in team_current_roles:
                if team_current_roles.count(i) > 1:
                    all_agents_in_disagreement += 1
            print("all_agents_in_disagreement", all_agents_in_disagreement)

            prob = 0.1 + (0.3 * self.common_ground) + ( (0.5/len(self.team_members)) * all_agents_in_disagreement ) 
            print("probability of conflict", prob)

            conflict = np.random.binomial(1, prob)
            

            # append conflict to a team dictionary
        
            self.model.team_conflict[self.team_id] = [conflict]
        else:
            pass

        print("This is the CONFLICT for", self.team_id, " :", self.model.team_conflict[self.team_id])
            
    def conflict_resolution(self):
        
        if self.team_id not in self.model.team_conflict_resolution:
            if self.model.team_conflict[self.team_id] == [1]:
                
                #if there is a conflict we can try to solve it

                baseline_probability = 0.1
                num_level3_agents = self.team_cps.count(3)
                if num_level3_agents > 0:
                    baseline_probability = 0.6 + ( 0.2/len(self.team_cps) ) * num_level3_agents
                num_level4_agents = self.team_cps.count(4)
                if num_level4_agents > 0:
                    baseline_probability = 0.8 + ( 0.2/len(self.team_cps) ) * num_level4_agents

                print("probability of conflict resolution", baseline_probability)
                conflict_solution = np.random.binomial(1, baseline_probability)
            
                self.model.team_conflict_resolution[self.team_id] = [conflict_solution]

                # Conflict of team is changed to 0 if resolution is 1
                if conflict_solution == 1:
                    self.model.team_conflict[self.team_id] = [0] # CHANGE CONFLICT to 0
                    print("Conflict resolution successful")
            elif self.model.team_conflict[self.team_id] == [0]:
                self.model.team_conflict_resolution[self.team_id] = [0]
        else:
            pass

        print("This is the CONFLICT RESOLUTION for", self.team_id, " :", self.model.team_conflict_resolution[self.team_id])


    def adjust_solution_for_disagreement(self):
        compromise_or_not = np.random.binomial(1, 0.5) # if 1 then compromise if 0 then not
        if len(self.agents_with_same_role) > 1:
            self.role_disagreement = 1 # if there are more than 1 agents with the same role then that agents is in disagreement

            both_solutions = list( [self.model.team_solutions[self.team_id][agent] for agent in self.agents_with_same_role] )
            print("this is both_solutions",both_solutions)
            both_solutions = unnest_list(both_solutions)
            print("this is both_solutions, iterated by unnest_list",both_solutions)
            print("this is both solution for the agents in the same role", both_solutions)
            if compromise_or_not == 1:
                self.model.team_solutions[self.team_id][self.unique_id] = np.mean(both_solutions)
                print("compromise, solution updated", self.model.team_solutions[self.team_id][self.unique_id])
            elif compromise_or_not == 0:
                if self.common_ground == 1:
                    self.model.team_solutions[self.team_id][self.unique_id] = max(both_solutions)
                    print("no compromise but common_ground, solution updated", self.model.team_solutions[self.team_id][self.unique_id])
                elif self.common_ground == 0:
                    random_solution = np.random.choice(both_solutions)
                    self.model.team_solutions[self.team_id][self.unique_id] = random_solution
                    print("no compromise no common_ground, solution updated", self.model.team_solutions[self.team_id][self.unique_id])

    # evaluating and comparing solutions 
    def evaluate(self):
        # print the solutions of all agents in the team
        

        all_round_team_solutions = list( self.model.team_solutions[self.team_id].values() )
        all_round_team_solutions = unnest_list(all_round_team_solutions) #unnest list from dictionary
        print("all_round_team_solutions before removal of duplicates", all_round_team_solutions)

        all_round_team_solutions = remove_duplicates(all_round_team_solutions) # removing duplicates
        print("all_round_team_solutions after removal", all_round_team_solutions)
       
        if self.common_ground == 1:
            team_best_solution = max(all_round_team_solutions)
        else:
            team_best_solution = random.choice(all_round_team_solutions)
        print("team_best_solution before appending", team_best_solution)
        

        # evaluate all solutions in the team_solutions dictionary indexed by team_id and store the highest in a model class variable called best_solution indexed by team_id
    
        if self.team_id not in self.model.best_solution:
            self.model.best_solution[self.team_id] = [team_best_solution]
        elif len(self.model.best_solution[self.team_id]) < self.model.schedule.p_round + 1:
            self.model.best_solution[self.team_id].append(team_best_solution)
        else:
            pass

        # print the team id and the best solution in the same line
        print("Common_ground", self.model.team_common_ground[self.team_id], "This is the solution picked for team", self.team_id, "problem round", self.model.schedule.p_round, " :", self.model.best_solution[self.team_id])


    # Resetting variables 
    # before running next generation of solutions and evaluations
    # But most importantly KEEP best_solutions and 
    # might use data collector to collect all team solutions 
    # or i can just create a secondary variable which stores all_team_solutions then append them to the global solutions where all solutions are stored
    def reset_variables(self):

        self.model.team_solutions = {}
        
        self.model.team_common_ground= {}

        self.model.team_current_roles = {}

        self.model.team_conflict = {}

        self.model.team_conflict_resolution = {}

    def reset_variables2(self):

        for key in self.model.team_solutions:
            self.model.team_solutions[key] = []
        
        for key in self.model.team_common_ground:
                self.model.team_common_ground[key] = []

    def reset_variables_per_round(self):
    
        self.model.p_round = self.model.schedule.p_round
        for key in self.model.team_solutions:
            self.model.team_solutions[key] = []
        
        for key in self.model.team_common_ground:
            self.model.team_common_ground[key] = []
        
        

    def test_loop(self):
        print("This is the loop")

    def test_loop2(self):
        print("This is the loop 2")

    # // STAGES //
    def stage_one(self):
        self.reset_team()
        self.join_teams3()
        self.get_correct_role()
        self.grab_team_cps()
    
    
    def stage_two(self):
        print("self.model.schedule.p_round", self.model.schedule.p_round)
        self.establish_common_ground()
        print("DEBUGGIN Agent", self.unique_id, "current role", self.current_role, "correct role", self.correct_role, "common ground", self.common_ground)
        
        self.getting_roles()

    def stage_three(self):
        self.find_agents_with_same_role()
        self.problem_loop()
        self.conflict()    
    
    def stage_four(self):
        self.adjust_solution_for_disagreement()
        self.conflict_resolution()

    def stage_five(self):
        self.evaluate()

    def stage_six(self):
        self.reset_variables()

    def stage_seven(self):
        self.test_loop2()
    
    def step(self):
        self.stage_one()
        for i in range(self.model.rounds):
            self.stage_two()

    

        
       

# Team Model

In [382]:
# Creating the model
class TeamModel(Model):
    def __init__(self, num_agents, rounds, problem_complexity_distribution):
        self.num_agents = num_agents
        self.rounds = rounds

        self.schedule = CustomStagedActivation(self, stage_list=["stage_one", "stage_two", "stage_three", "stage four", "stage_five"], num_rounds=self.rounds)
        self.schedule2 = StagedActivation(self, stage_list=["stage_two"])
        
        self.teams = {}
        self.agent_team_dict = {}
        self.groups = []

        self.problem_complexity_distribution = problem_complexity_distribution
        self.problem_complexity = 0

        self.team_current_roles = {}
        self.team_common_ground = {}
        self.team_conflict = {}
        self.team_conflict_resolution = {}

        self.team_solutions = {}
        self.best_solution = {}
        
        self.p_round = 0
        
        for i in range(self.num_agents):
            agent = TeamAgent(i, self)
            self.schedule.add(agent)

    # Creating team lists
    def create_teams(self):
        # Creating random teams of 2-5 agents based on the number of agents
        numbers = []
        x = 0
        while len(numbers) < self.num_agents:
            repetitions = random.randint(2, 5)
            numbers.extend([x] * repetitions)
            x += 1
        numbers = numbers[:self.num_agents]  # Trim the list to the desired number of agents
        if numbers.count(numbers[self.num_agents - 1]) < 2:
            numbers.remove(numbers[self.num_agents - 1])
            possible_teams = []
            for i in set(numbers):
                if numbers.count(i) < 5:
                    possible_teams.append(i)
            numbers.append(random.choice(possible_teams))
        

        random.shuffle(numbers) 

        self.groups = numbers

        # Creating the roles such that
        # each team has 2 to 5 roles specified by 1:n

        all_agents = list( range(0, self.num_agents) )
        self.teams = {
            "agent_id": all_agents,
            "team_id": self.groups
        }

        self.agent_team_dict = {agent_id: team_id for agent_id, team_id in zip(self.teams['agent_id'], self.teams['team_id'])}
        

    
    # creating a problem which is then used for solving
    def generate_problem(self): 
        self.problem_complexity = np.random.choice([1,2,3], p=self.problem_complexity_distribution)


    def get_agent_by_id(self, agent_id):
        return self.schedule._agents[agent_id]  # Accessing agent using its unique_id
    
    # A function which checks
    def collect_teams(self):
        print(self.teams)
        print(self.agent_team_dict)

    def reset_variables(self):
        self.teams = {}
        self.agent_team_dict = {}
        self.groups = []

        self.team_current_roles = {}
        self.team_common_ground = {}
        self.team_team_conflict = {}

        self.team_solutions = {}
        self.best_solution = {}

        self.p_round = 0

    def step(self):
        self.reset_variables()
        self.create_teams()
        self.generate_problem()
        self.schedule.step()
        # create a loop which initiates the loop for self.rounds
        self.collect_teams()




# Run the Model

In [383]:

model = TeamModel(num_agents=20, rounds=10, problem_complexity_distribution=[0.333, 0.333, 0.334])

for i in range(2):
    model.step()



agent_id: 0 Team_id: 4 Team_members: [0, 2, 8, 10, 18]
Your correct role is: 0
This is your own cps: 3
This is your weight 1.8173099320467703
This is the cps of whole team: [3, 1, 4, 3, 3]
agent_id: 1 Team_id: 2 Team_members: [1, 6, 13, 17]
Your correct role is: 0
This is your own cps: 3
This is your weight 1.9129145825199119
This is the cps of whole team: [3, 3, 2, 0]
agent_id: 2 Team_id: 4 Team_members: [0, 2, 8, 10, 18]
Your correct role is: 1
This is your own cps: 1
This is your weight 1.229999076366375
This is the cps of whole team: [3, 1, 4, 3, 3]
agent_id: 3 Team_id: 0 Team_members: [3, 5, 19]
Your correct role is: 0
This is your own cps: 1
This is your weight 1.1643120662003343
This is the cps of whole team: [1, 1, 2]
agent_id: 4 Team_id: 3 Team_members: [4, 11, 14, 15]
Your correct role is: 0
This is your own cps: 2
This is your weight 1.4664319008465174
This is the cps of whole team: [2, 2, 3, 2]
agent_id: 5 Team_id: 0 Team_members: [3, 5, 19]
Your correct role is: 1
This is 